In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import dlc_practical_prologue as prologue

Generate pairs

In [3]:
N = 1000 #nb of pairs

#generate pairs
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

In [4]:
#normalize the input
train_input/=255
test_input/=255

In [5]:
print_shapes_Net = False

In [6]:
class Siamese_net(nn.Module):
    def __init__(self):
        super(Siamese_net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, 10)
        self.fc2 = nn.Linear(20, 2)

    def forward(self, data):
        if print_shapes_Net:
            print("initial", data.shape) #100 2 14 14
            
        final_layer = []
        for i in range(2):
            x = data[:,i,:,:]
            len0 = x.shape[0]
            x = torch.reshape(x, (len0, 1, 14, 14))
            
            if print_shapes_Net:
                print("X START",x.shape) 
            
            x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
            if print_shapes_Net:
                print("conv1",x.shape) 
                
            x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
            if print_shapes_Net:
                print("conv2",x.shape)
            
            x = F.relu(self.fc1(x.view(-1, 256)))
            if print_shapes_Net:
                print("fc1",x.shape) 
                
            final_layer.append(x)
            
        #final_layer = torch.abs(final_layer[1] - final_layer[0])
        final_layer = torch.cat((final_layer[0], final_layer[1]), 1)
                
        final_layer = self.fc2(final_layer)
        if print_shapes_Net:
            print("final",final_layer.shape) 
            
        return final_layer

In [7]:
def train_model(model, train_input, train_target, mini_batch_size, lr, nb_epoch):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr)
    
    for e in range(nb_epoch):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            
            sum_loss = sum_loss + loss.item()
        #print(e, sum_loss)

In [8]:
def compute_nb_errors_targets(model, input, target):
    nb_errors = 0
    output = model(input)
    _, predicted_target = output.max(1) #max probabilities of target
    
    for b in range(1000):
        if target[b,int(predicted_target[b])] <= 0:
            nb_errors = nb_errors + 1
            
    return nb_errors

In [9]:
#train_target[1000,1]

new_train_target = torch.empty(1000,2)
new_test_target = torch.empty(1000,2)
for i in range(1000):
    if train_target[i] == 1 :
        new_train_target[i,0] = 0
        new_train_target[i,1] = 1
        
    else:
        new_train_target[i,0] = 1
        new_train_target[i,1] = 0
        
    if test_target[i] == 1:
        new_test_target[i,0] = 0
        new_test_target[i,1] = 1
        
    else:
        new_test_target[i,0] = 1
        new_test_target[i,1] = 0

#### Tests

In [10]:
model = Siamese_net()
lr = 0.001
mini_batch_size = 100
nb_epoch = 25

train_model(model, train_input, new_train_target, mini_batch_size, lr, nb_epoch)
print("LR =", lr)

nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
print('train error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                  nb_train_errors, train_input.size(0)))
nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
print('test error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))

LR = 0.001
train error Siamese_net 6.80% 68/1000
test error Siamese_net 13.60% 136/1000


In [10]:
model = Siamese_net()
lr = 0.001
mini_batch_size = 100
nb_epoch = 25

train_model(model, train_input, new_train_target, mini_batch_size, lr, nb_epoch)
print("LR =", lr)

nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
print('train error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                  nb_train_errors, train_input.size(0)))
nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
print('test error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))

LR = 0.001
train error Siamese_net 6.90% 69/1000
test error Siamese_net 13.40% 134/1000


#### Choice of parameter for Siamese_net

In [101]:
# lr = 0.001 --> no overfitting
for lr in [0.0005, 0.001,0.005, 0.01, 0.05, 0.1, 0.5]:
    model = Siamese_net()
    mini_batch_size = 100
    nb_epoch = 25

    train_model(model, train_input, new_train_target, mini_batch_size, lr, nb_epoch)
    print("LR =", lr)
    
    nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
    print('train error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                      nb_train_errors, train_input.size(0)))
    nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
    print('test error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))

LR = 0.0005
train error Net 14.50% 145/1000
test error Net 18.50% 185/1000
LR = 0.001
train error Net 7.00% 70/1000
test error Net 15.30% 153/1000
LR = 0.005
train error Net 0.00% 0/1000
test error Net 14.10% 141/1000
LR = 0.01
train error Net 0.00% 0/1000
test error Net 14.20% 142/1000
LR = 0.05
train error Net 46.40% 464/1000
test error Net 47.30% 473/1000
LR = 0.1
train error Net 46.40% 464/1000
test error Net 47.30% 473/1000
LR = 0.5
train error Net 46.40% 464/1000
test error Net 47.30% 473/1000


#### Final test

In [102]:
mini_batch_size = 100
nb_epoch = 25
lr = 0.001

for i in range(10):
    model = Siamese_net()
    train_model(model, train_input, new_train_target, mini_batch_size, lr, nb_epoch)
    print("i =", i)

    nb_train_errors = compute_nb_errors_targets(model, train_input, new_train_target)
    print('train error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_train_errors) / train_input.size(0),
                                                      nb_train_errors, train_input.size(0)))
    nb_test_errors = compute_nb_errors_targets(model, test_input, new_test_target)
    print('test error Siamese_net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                    nb_test_errors, test_input.size(0)))

i = 0
train error Net 7.20% 72/1000
test error Net 15.20% 152/1000
i = 1
train error Net 5.20% 52/1000
test error Net 13.90% 139/1000
i = 2
train error Net 6.00% 60/1000
test error Net 14.10% 141/1000
i = 3
train error Net 4.50% 45/1000
test error Net 14.60% 146/1000
i = 4
train error Net 5.50% 55/1000
test error Net 15.70% 157/1000
i = 5
train error Net 6.90% 69/1000
test error Net 15.10% 151/1000
i = 6
train error Net 10.20% 102/1000
test error Net 16.20% 162/1000
i = 7
train error Net 7.20% 72/1000
test error Net 15.70% 157/1000
i = 8
train error Net 10.90% 109/1000
test error Net 17.10% 171/1000
i = 9
train error Net 9.90% 99/1000
test error Net 16.10% 161/1000
